# Les 2:

Voor deze les hebben we `bagofwords.py` gemaakt. Dit script kan gegeven een `.txt` bestand met daarin per regel een document deze omzetten in een reeks getalwaarden, voor deze omzetting is er keuze uit: multi-hot encoding, count encoding en TF-IDF-encoding. De resultaten hiervan kunnen omgezet worden in een `.bow` bestand. Om de functionaliteiten van dit script te demonstreren volgt in dit notebook een demo.

## .txt bestanden verkrijgen:

Om `bagofwords.py` te demonstreren hebben we een aantal pubmed abstracts + titels nodig in een `.txt` bestand. Deze kunnen we downloaden met onderstaande code. Voor de niet kanker bestanden hebben gekozen voor artikelen over voeding.

In [ ]:
from Bio import Entrez

Entrez.email = "voorbeeld.mailadres@mail.com" # verander naar je eigen email als je dit na wilt doen

def get_pubmed_abstracts(query, max_results=100):
    handle = Entrez.esearch(db="pubmed", term=query, retmax=max_results)
    record = Entrez.read(handle)
    id_list = record["IdList"]

    abstracts = []
    for pubmed_id in id_list:
        handle = Entrez.efetch(db="pubmed", id=pubmed_id, rettype="abstract", retmode="text")
        abstract_text = handle.read().replace("\n", "")
        abstracts.append(abstract_text)
    return abstracts

kanker_docs = get_pubmed_abstracts("cancer", max_results=50)
niet_kanker_docs = get_pubmed_abstracts("nutrition", max_results=50)

Nu gaan we deze variabelen wegschrijven naar twee `.txt` bestanden en kijken hoe groot deze zijn

In [ ]:
with open("kanker.txt", "w", encoding = "utf-8") as f:
    for doc in kanker_docs: 
        f.write(doc.replace("\n", " ") + "\n")

with open("niet_kanker.txt", "w", encoding="utf-8") as f:
    for doc in niet_kanker_docs:
        f.write(doc.replace("\n", " ") + "\n")

In [18]:
with open("kanker.txt", "r", encoding="utf-8") as f:
    lines = len(f.readlines())
    print(f"Kanker abstracts: {lines}")

with open("niet_kanker.txt", "r", encoding="utf-8") as f:
    lines = len(f.readlines())
    print(f"Niet kanker abstracts: {lines}")

Kanker abstracts: 50
Niet kanker abstracts: 50


Beide `.txt` bestanden bevatten 50 titels + abstracts.

## Bag of words encoding:

Nu we de `.txt` bestanden hebben is het tijd om de bag of words encoding hierop toe te passen. In ons script is keuze uit: multi-hot encoding, count, en tf-df. 

Allereerst gaan we count bestanden maken:

In [31]:
!python bagofwords.py kanker.txt niet_kanker.txt --encoding count --minfreq 100

Bag of words met het bestand: kanker.txt
Vocabulary: {'ff', 'ys', 'lo', 'nutrition', 'to', 'å', 'ic', '_', 'pres', 'tion', '±', 'ital', '0', '6', 'cl', "'", 'ations', '1', '10.', 'os', 'anal', 'y', 'cancer', 'pati', 'mon', 'health', 'doi:', 'form', '8', 'pl', 'ph', 'interest', 'tiv', 'der', '12', 'γ', ').', 'ch', 'medical', 'enc', 'ers', 's', 've', 'ght', '–', 'a.', 'id', 'mic', 'part', '68', 'ential', 'nutri', 'j', 'ap', 'ang', '25', 'tive', 'é', 'sci', 'f', 'inter', 'gn', 'department', 'th', 'this', 'od', 'university,', 'ac', 'n', 'information', 'medic', 'ical', 'ú', 'ary', 'ation', 'with', 'patient', 'dis', 'ence', '∼', 'v', 'ha', 's.', 'de', 'gr', 'sion', 'con', 'po', 'ivers', 'fo', 'anc', ')', 'x', '€', 'val', 'all', 'as', 'hos', '10', 'su', 'par', 'lic', '(1', 'iz', 'le', 'formation', 'ong', 'vi', 'o', 's:', 'st', '@', 'ated', 'á', 'ar', 'clin', 'll', 'um', 'e', 'is', 'the', '20', 'ess', 'man', 'ty', 'res', '≥', 'il', 'sy', 'ance', '(1),', 'car', '̇', 'ub', 'ase', '(4', 'ra', 'ch

Nu gaan we de multi-hot optie gebruiken:

In [36]:
!python bagofwords.py kanker.txt niet_kanker.txt --encoding multi-hot --minfreq 100

Bag of words met het bestand: kanker.txt
Vocabulary: {'w', 'information', 'ure', 'eff', '’', '%', 'fe', 'ancer', 'ght', 'con', 'ğ', 'hosp', 'gh', 'er', 'soci', 'ol', '–', 'are', '9', 'it', 're', 'y.', ')', 'as', 'u', 'c', 'e', 'um', 'ative', 'os', 'med', '(2', 'iv', '-', 'ó', 'enc', 'car', ';', 'chin', 'ap', 'by', 'ã', 'ec', 'ion', 'β', 'æ', '<', 'ne', 'uc', 'nutri', 'de', 'the', 'int', 's,', '_', 'on', 'ha', 'mun', 'ter', 'conf', 'ty', 'cell', 'è', 'this', 'fin', 'interest', '@', 'su', 'pe', 'mon', 'we', 'x', 'ital', 'ations', 'z', 'og', 'q', '3', 'pmid:', 'par', 'patient', 'ub', 'ation', 'ear', ').', '∼', 'ess', 'stud', 'ob', 'ex', 'ch', 'man', 'y,', 'department', '13', 'mic', 's:', 'ing', 'ine', 'fic', 'ph', 'pres', 'μ', 'tion', 'olog', 'earch', 'n', 'ú', 'cancer', 'ro', 'pati', 'd', 'is', 'pm', 'ab', 'and', '(', '(1)', 'medic', '5', '>', '2', 'gro', 'ver', 'tive', 'ar', 'bi', '12', 'am', '/', 'associ', 'ly', 'all', '7', '1.', ',', 'ci', 'health', 'ary', 'es.', 'anal', '0', 'ed', '€

En als laatst de TF-IDF optie:

In [37]:
!python bagofwords.py kanker.txt niet_kanker.txt --encoding tfidf --minfreq 100

Bag of words met het bestand: kanker.txt
Vocabulary: {'all', 'for', 'es,', 'i:', 'ys', 'str', 'ts', 'by', 'su', 'nutri', 'ology,', 'was', 'st', 'fin', 'fo', 'au', 'low', 'ted', 'ure', 'os', 'um', 'ing', 'pm', 'med', 'ch', 'mun', 'ob', '@', 'high', 'si', 'be', 'æ', 'pres', 'nut', 'h', 'k', 'out', '(2', 'll', 'pot', 'information', 'cent', 'ex', 'cl', 'tr', 'l', 'β', 're', 'pre', 'ment', 'medic', 'patient', 'al', 'res', '+', 'y,', 'ic', 'cell', 'medicine', 'er', 'ong', '≤', 'pl', 'ud', 'ance', 'i', 'and', 'ght', 'this', 'fe', 'me', 'ub', 'up', 'qu', 'ear', 'enc', 'he', 'no', 'é', 'sy', 'og', '0.', 'η', 'p', 'univers', '=', 'us', 'ent', '16', 'pa', 'ol', '20', 'å', '12', 'po', ')department', 'ab', 's', 'lo', 'chin', 'soci', 'ations', '“', 'ther', '),', 'ap', 'sign', 'pati', 'do', 'at', 'ans', 'th', '≥', '"', 'ate', 'r', '(4', 'ase', 'so', 'id', 'ha', 'ed', 'wi', 'ci', 's.', 'duc', '2025.', 'conflict', 'un', 'ff', 'ess', 'ant', 'und', 'ø', 'α', 'μ', 'inc', 'clin', 'medical', '_', 'china.', 

## Machine learning met sklearn:

Met sklearn gaan we kijken naar de verschillende vormen van bag of words. We splitsen de data op in twee klassen: kanker en niet kanker.

### Count encoding:

We gaan beginnen met de count encoding.

In [1]:
# De bestanden inladen als numpy arrays:
import numpy as np

with open("kanker_count.bow", "r", encoding="utf8") as f:
    data = []
    for line in f:
        line = line.strip()
        if line.startswith("[") and line.endswith("]"):
            data.append(np.array(eval(line)))
            kanker_count_array = np.array(data)

with open("niet_kanker_count.bow", "r", encoding="utf8") as f:
    data = []
    for line in f:
        line = line.strip()
        if line.startswith("[") and line.endswith("]"):
            data.append(np.array(eval(line)))
            niet_kanker_count_array = np.array(data)

In [2]:
# Labels toevoegen:
labels_kanker = np.ones(len(kanker_count_array))
labels_nietkanker = np.zeros(len(niet_kanker_count_array))

In [3]:
# De twee samenvoegen:
X_count = np.vstack([kanker_count_array, niet_kanker_count_array])
y_count = np.concatenate([labels_kanker, labels_nietkanker])

In [4]:
# Splitsen in training en test data:
from sklearn.model_selection import train_test_split
X_train_count, X_test_count, y_train_count, y_test_count = train_test_split(
    X_count, y_count, test_size=0.2, random_state=42)

In [5]:
# De classifier trainen:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

count_model = MultinomialNB()
count_model.fit(X_train_count, y_train_count)

,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


In [6]:
# Voorspellen met de test data:
y_pred_count = count_model.predict(X_test_count)

### Multi-hot encoding:

Als volgende gaan we sklearn gebruiken met de multi-hot encoding `.bow` bestanden.

In [7]:
# De bestanden inladen als numpy arrays:
with open("kanker_multihot.bow", "r", encoding="utf8") as f:
    data = []
    for line in f:
        line = line.strip()
        if line.startswith("[") and line.endswith("]"):
            data.append(np.array(eval(line)))
            kanker_multihot_array = np.array(data)

with open("niet_kanker_multihot.bow", "r", encoding="utf8") as f:
    data = []
    for line in f:
        line = line.strip()
        if line.startswith("[") and line.endswith("]"):
            data.append(np.array(eval(line)))
            niet_kanker_multihot_array = np.array(data)

In [8]:
# Labels toevoegen:
labels_kanker = np.ones(len(kanker_multihot_array))
labels_nietkanker = np.zeros(len(niet_kanker_multihot_array))

In [9]:
# De twee samenvoegen:
X_multihot = np.vstack([kanker_multihot_array, niet_kanker_multihot_array])
y_multihot = np.concatenate([labels_kanker, labels_nietkanker])

In [10]:
# Splitsen in training en test data:
X_train_multihot, X_test_multihot, y_train_multihot, y_test_multihot = train_test_split(
    X_multihot, y_multihot, test_size=0.2, random_state=42)

In [11]:
# De classifier trainen:
multihot_model = MultinomialNB()
multihot_model.fit(X_train_multihot, y_train_multihot)

,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


In [12]:
# Voorspellen met de test data:
y_pred_multihot = multihot_model.predict(X_test_multihot)

### TF-IDF encoding:

Tot slot gaan we sklearn gebruiken met de TF-IDF `.bow` bestanden.

In [13]:
# De bestanden inladen als numpy arrays:
with open("kanker_tfidf.bow", "r", encoding="utf8") as f:
    data = []
    for line in f:
        line = line.strip()
        if line.startswith("[") and line.endswith("]"):
            data.append(np.array(eval(line)))
            kanker_tfidf_array = np.array(data)

with open("niet_kanker_tfidf.bow", "r", encoding="utf8") as f:
    data = []
    for line in f:
        line = line.strip()
        if line.startswith("[") and line.endswith("]"):
            data.append(np.array(eval(line)))
            niet_kanker_tfidf_array = np.array(data)

In [14]:
# Labels toevoegen:
labels_kanker = np.ones(len(kanker_tfidf_array))
labels_nietkanker = np.zeros(len(niet_kanker_tfidf_array))

In [15]:
# De twee samenvoegen:
X_tfidf = np.vstack([kanker_tfidf_array, niet_kanker_tfidf_array])
y_tfidf = np.concatenate([labels_kanker, labels_nietkanker])

In [16]:
# Splitsen in training en test data:
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(
    X_tfidf, y_tfidf, test_size=0.2, random_state=42)

In [17]:
# De classifier trainen:
tfidf_model = MultinomialNB()
tfidf_model.fit(X_train_tfidf, y_train_tfidf)

,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


In [18]:
# Voorspellen met de test data:
y_pred_tfidf = tfidf_model.predict(X_test_tfidf)

### Effect verschillende encodings op sklearn

Nu gaan we kijken of de gekozen bag of words representatie effect heeft op de voorspellingen van het model. Dit gaan we doen aan de hand van een aantal evualatie waarden.

In [19]:
print("Count encoding:")
print("Accuracy:", accuracy_score(y_test_count, y_pred_count))
print("Precision:", precision_score(y_test_count, y_pred_count))
print("Recall:", recall_score(y_test_count, y_pred_count))
print("f1:", f1_score(y_test_count, y_pred_count), "\n")

print("Multi-hot encoding:")
print("Accuracy:", accuracy_score(y_test_multihot, y_pred_multihot))
print("Precision:", precision_score(y_test_multihot, y_pred_multihot))
print("Recall:", recall_score(y_test_multihot, y_pred_multihot))
print("f1:", f1_score(y_test_multihot, y_pred_multihot), "\n")

print("TF-IDF encoding:")
print("Accuracy:", accuracy_score(y_test_tfidf, y_pred_tfidf))
print("Precision:", precision_score(y_test_tfidf, y_pred_tfidf))
print("Recall:", recall_score(y_test_tfidf, y_pred_tfidf))
print("f1:", f1_score(y_test_tfidf, y_pred_tfidf))

Count encoding:
Accuracy: 0.75
Precision: 0.8888888888888888
Recall: 0.6666666666666666
f1: 0.7619047619047619 

Multi-hot encoding:
Accuracy: 0.95
Precision: 0.9230769230769231
Recall: 1.0
f1: 0.96 

TF-IDF encoding:
Accuracy: 0.9
Precision: 1.0
Recall: 0.8333333333333334
f1: 0.9090909090909091


Als we kijken naar de verschillende evaluatie waarden voor de modellen is er te zien dat elk model anders scoort op de verschillende elementen. Sommigen liggen wel redelijk dicht bij elkaar zoals de precision en f1 scores van het model dat de multi-hot encoding heeft gebruikt liggen dichtbij de precision en f1 scores van het model dat de TF-IDF encoding heeft gebruikt.

Het model dat de count-encoding heeft gebruikt scoort op elke waarde lager dan de modellen die de multi-hot en TF-IDF encoding hebben gebruikt. 

Verder valt het op dat het model dat de TF-IDF encoding heeft gebruikt een precision van 1.0 model scoort, dit houdt in dat dit model *geen* false positieve heeft, alleen maar true positives.

Hieruit kunnen we dus concluderen dat afhankelijk van welke encoding wordt gekozen dit effect kan hebben op de resultaten van de classifier, wanneer je gaat werken aan een classificatie probleem is het belangrijk om na te gaan welke vorm van encoding je wilt gaan gebruiken.